In [17]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [18]:
data=pd.read_csv("RaleighCityOriginalDataFY20.csv",)

In [19]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,adopted
1,2,1,budget
2,3,1,fy
3,4,1,strengthening
4,5,1,our


In [20]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,adopted
1,1,budget
2,1,fy
3,1,strengthening
4,1,our
5,1,foundation
6,1,in
7,1,workforce
8,1,operations
9,1,and


In [21]:
data.describe()

,Unnamed: 0,page_number
count,72573.00000,72573.000000
mean,36287.00000,162.375815
std,20950.16488,90.118814
min,1.00000,1.000000
25%,18144.00000,85.000000
50%,36287.00000,171.000000
75%,54430.00000,231.000000
max,72573.00000,317.000000


In [22]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [25]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [26]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [28]:
#creating tokens
import random
text_data=[]
with open('RaleighCityOriginalDataFY20.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['30",2,"city']
['42",2,"nancy']
['180",4,"a']
['754",6,"proposal']
['774",6,"budget']
['897",7,"unique']
['1064",7,"grants']
['1100",7,"rate']
['1416",8,"the']
['1423",8,"structure']
['1579",8,"annual']
['1591",8,"to']
['1653",9,"year']
['1827",9,"for']
['1843",9,"supervisor']
['1967",10,"and']
['1975",10,"community']
['2161",10,"to']
['2394",11,"allocated']
['2795",12,"general']
['2810",12,"housing']
['2911",13,"for']
['3047",13,"city']
['3067",13,"on']
['3748",15,"and']
['3867",15,"is']
['4141",15,"noted']
['4248",15,"and']
['4295",15,"december']
['4345",16,"by']
['4380",16,"december']
['4391",16,"budget']
['4591",16,"of']
['4733",17,"city']
['4828",18,"a']
['5028",19,"policies']
['5232",19,"yearend']
['5282",19,"capital']
['5321",19,"ordinance']
['5526",19,"requirements']
['5600",19,"which']
['5649",20,"at']
['5692",20,"fund']
['5854",21,"performance']
['6053",21,"for']
['6179",23,"also']
['6287",24,"between']
['6348",24,"raleigh']
['6439",25,"university']
['6733",25,"management']


['42936",196,"the']
['42953",196,"required']
['43027",196,"final']
['43084",196,"server']
['43092",196,"identification']
['43135",197,"project']
['43232",197,"of']
['43350",197,"categories']
['43529",197,"permit']
['43696",198,"calculated']
['43969",198,"and']
['44308",199,"systems']
['44326",199,"commercial']
['44333",199,"commercial']
['44384",199,"trade']
['44399",199,"cpi']
['44410",200,"prior']
['44453",200,"land']
['44496",200,"cpi']
['44544",200,"recorded']
['44612",200,"site']
['44664",201,"permit']
['44843",201,"greater']
['44959",201,"per']
['44981",201,"street']
['45110",202,"greater']
['45284",202,"the']
['45305",202,"with']
['45355",202,"feetper']
['45470",202,"minor']
['45494",202,"permit']
['45595",202,"fee']
['45659",202,"per']
['46134",203,"at']
['46195",203,"base']
['46371",204,"flat']
['46473",204,"issue']
['46579",205,"linear']
['46755",206,"slope']
['46784",206,"enr']
['46894",206,"inspections']
['47049",207,"process']
['47078",208,"and']
['47119",208,"needed']
['4

In [29]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [30]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.008*"41315",192,"visual" + 0.008*"55909",239,"only" + 0.008*"48452",211,"and" + 0.008*"11948",56,"fy" + 0.008*"67045",294,"resources" + 0.008*"44959",201,"per" + 0.008*"32994",159,"potential" + 0.008*"25515",121,"departments" + 0.008*"51388",218,"splash" + 0.008*"70638",308,"is"
Topic: 1 word: 0.008*"56536",242,"wilmington" + 0.008*"13153",61,"appropriation" + 0.008*"14249",69,"positive" + 0.008*"8295",33,"shown" + 0.008*"66531",292,"of" + 0.008*"27364",134,"growth" + 0.008*"42437",195,"use" + 0.008*"32635",158,"equipment" + 0.008*"48009",210,"construction" + 0.008*"15972",78,"of"
Topic: 2 word: 0.008*"18491",86,"revenues" + 0.008*"32084",156,"the" + 0.008*"55280",236,"week" + 0.008*"46134",203,"at" + 0.008*"65070",288,"employment" + 0.008*"29559",144,"yr" + 0.008*"31104",153,"buffaloe" + 0.008*"41357",192,"accept" + 0.008*"38236",179,"and" + 0.008*"41010",190,"a"
Topic: 3 word: 0.008*"22975",109,"three" + 0.008*"21823",102,"to" + 0.008*"67204",294,"system" + 0.008*"71

In [31]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)